In [ ]:
# Cell 1: Importera alla bibliotek vi behöver

import pandas as pd              # För att läsa och hantera tabell-data (DataFrames)
import numpy as np               # För numeriska beräkningar

from sklearn.model_selection import train_test_split   # Delar data i train/test
from sklearn.neighbors import KNeighborsClassifier     # k-NN klassificerare
from sklearn.preprocessing import MinMaxScaler         # Normalisering till [0,1]
from sklearn.metrics import confusion_matrix, accuracy_score  # Utvärderingsmått

import matplotlib.pyplot as plt   # För visualiseringar (diagram)


In [ ]:
# Cell 2: Läs in datasetet och skapa features (X) och target (y)

# Läs in student-por.csv (portugisiska elever)
df = pd.read_csv("student-por.csv", sep=";")

# Välj alla numeriska kolumner i datan
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Features X = alla numeriska kolumner UTOM G3 (slutbetyget)
X = df[numeric_cols].drop(columns=["G3"])

# Target y = slutbetyg G3 omvandlat till tre klasser:
# 0–9   -> "low"
# 10–14 -> "medium"
# 15–20 -> "high"
bins = [-1, 9, 14, 20]                # Gränser för intervallen
labels = ["low", "medium", "high"]    # Klassnamn
y = pd.cut(df["G3"], bins=bins, labels=labels)

# Kolla så att vi har rimligt med observationer i varje klass
print("Fördelning av klasser i y:")
print(y.value_counts())
print("\nForm på X (antal rader, antal features):", X.shape)


In [ ]:
# Cell 3: Funktion som gör EN k-NN-körning och returnerar confusion matrix + accuracy

def run_knn(X, y, k, test_size, normalize=False, random_state=42):
    """
    Gör EN körning med k-NN.

    Parametrar:
    - X : features (predictors)
    - y : target (klasserna low/medium/high)
    - k : antal grannar i k-NN
    - test_size : andel av datan som ska användas som test (t.ex. 0.1, 1/3, 0.5)
    - normalize : True => normalisera X till [0,1] (MinMaxScaler)
    - random_state : används för train_test_split så att vi får samma split varje gång
                     (bra för att kunna jämföra resultaten)
    Returnerar:
    - cm : confusion matrix (3x3)
    - acc : accuracy (flyttal mellan 0 och 1)
    """

    # 1. Dela upp data i train och test
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=test_size,
        stratify=y,          # Behåll samma klassfördelning i train och test
        random_state=random_state
    )

    # 2. Normalisera om vi valt det
    if normalize:
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train = scaler.fit_transform(X_train)   # Lär sig min/max från träningsdatan
        X_test = scaler.transform(X_test)         # Använder samma skalning på testdatan
    else:
        # Om vi inte normaliserar, konverterar vi bara till numpy-arrayer (frivilligt)
        X_train = X_train.values
        X_test = X_test.values

    # 3. Skapa och träna k-NN-modellen
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    # 4. Prediktion på testdatan
    y_pred = knn.predict(X_test)

    # 5. Confusion matrix och accuracy
    cm = confusion_matrix(y_test, y_pred, labels=y.cat.categories)
    acc = accuracy_score(y_test, y_pred)

    return cm, acc


In [ ]:
# Cell 4: Kör alla 18 kombinationer och skriv ut confusion matrix + accuracy

# Tre olika k-värden (udda mellan 3 och 15)
k_values = [3, 7, 11]

# Tre olika train/test-splittar
splits = [
    (0.10, "train=90%, test=10%"),
    (1/3,  "train=2/3, test=1/3"),
    (0.50, "train=50%, test=50%")
]

# Lista där vi sparar alla resultat (för tabell och top 3 senare)
results = []

for k in k_values:
    for test_size, split_text in splits:
        # Två datatyper: originaldata och normaliserat [0,1]
        for normalize_flag, data_text in [
            (False, "originaldata"),
            (True,  "normaliserat [0,1]")
        ]:
            # Kör en k-NN-exekvering
            cm, acc = run_knn(
                X, y,
                k=k,
                test_size=test_size,
                normalize=normalize_flag,
                random_state=42   # samma slump-split för alla kombinationer
            )

            # Skriv ut resultatet
            print("--------------------------------------------------")
            print(f"k = {k}, data = {data_text}, split = {split_text}")
            print("Confusion matrix (rader/kolumner: low, medium, high):")
            print(cm)
            print(f"Accuracy: {acc:.4f}")

            # Spara resultatet i en lista
            results.append({
                "k": k,
                "data_typ": data_text,
                "split_text": split_text,
                "test_size": test_size,
                "normalize": normalize_flag,
                "accuracy": acc,
                "confusion_matrix": cm
            })


In [ ]:
# Cell 5: Skapa tabell med alla 18 körningar och ta fram de tre bästa

# Gör om results-listan till en DataFrame, men hoppa över själva confusion_matrix-objektet här
res_df = pd.DataFrame([
    {k: v for k, v in r.items() if k != "confusion_matrix"}
    for r in results
])

print("Alla 18 körningar (utan confusion matrices):")
display(res_df[["k", "data_typ", "split_text", "accuracy"]])

# Sortera efter accuracy (högst först) och ta de tre bästa körningarna
top3 = res_df.sort_values(by="accuracy", ascending=False).head(3)

print("\nTre bästa körningar (enligt accuracy):")
display(top3)


In [ ]:
# Cell 6: Visa confusion matrices för de tre bästa körningarna

print("Confusion matrices för de tre bästa körningarna:\n")

# Gå igenom de tre bästa raderna
for i, row in top3.reset_index(drop=True).iterrows():
    # Hitta motsvarande dict i results-listan
    best_cm = None
    for r in results:
        if (
            r["k"] == row["k"] and
            r["data_typ"] == row["data_typ"] and
            r["split_text"] == row["split_text"] and
            abs(r["accuracy"] - row["accuracy"]) < 1e-9
        ):
            best_cm = r["confusion_matrix"]
            break

    print("==============================================")
    print(f"Körning {i+1}:")
    print(f"k = {row['k']}, data = {row['data_typ']}, split = {row['split_text']}")
    print(f"Accuracy: {row['accuracy']:.4f}")
    print("Confusion matrix (low, medium, high):")
    print(best_cm)
